In [10]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"


In [11]:
def get_completion_from_messages(messages, 
                                 model="gpt-35-turbo-16k", 
                                 temperature=0, max_tokens=4000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
        engine="gpt-35-turbo-16k",
        
    )
    return response.choices[0].message["content"]

In [23]:
import json

delimiter = "####"

user_message = f"""
I want to create a web app
"""

system_message = """
You are a bot helping create Azure resources.

I will give you a json format with placeholders, the placeholders are with format **placeholder**.
You help replace the placeholders according to users' request or the default ones, and output the json for users.
The users' query will be delimited with \
#### characters.
You must follow these rules:
1. Everytime user request, just output the json, don't ask more questions.
2. Output only the json, nothing else, nothing else, nothing else!
3. If you don't have enought inputs, just use the default ones.

Your response should with format below:
{
    "id": "/subscriptions/937bc588-a144-4083-8612-5f9ffbbddb14/resourceGroups/**ResourceGroup**/providers/Microsoft.Web/sites/**WebAppName**?api-version=2022-03-01",       
    "payload": {
        "location": "**Location**",
        "properties": {
            "serverFarmId": "**AppServicePlanId**",
            "reserved": false,
            "isXenon": false,
            "hyperV": false,
            "siteConfig": {
                "netFrameworkVersion": "v4.6",
                "linuxFxVersion": "**Runtime**",
                "appSettings": [],
                "alwaysOn": true,
                "localMySqlEnabled": false,
                "http20Enabled": true
            },
            "scmSiteAlsoStopped": false,
            "httpsOnly": false
        }
    }
}

If ResourceGroup not provided, choose from below that best matches request
{
"id": "/subscriptions/937bc588-a144-4083-8612-5f9ffbbddb14/resourceGroups/Cupertino-MVP",
"location": "centralus",
"name": "Cupertino-MVP"
}
{
"id": "/subscriptions/937bc588-a144-4083-8612-5f9ffbbddb14/resourceGroups/GenevaWarmPathManageRG",
"location": "centralus",
"name": "GenevaWarmPathManageRG"
}

If WebAppName is not provided, use a random string with 10 characters, with prefix `app-`

If AppServicePlanId not provided, choose from below that best matches request
[{
    "id": "/subscriptions/937bc588-a144-4083-8612-5f9ffbbddb14/resourceGroups/houk-test/providers/Microsoft.Web/serverfarms/ASP-houktest-8820",
    "location": "Central US",
    "kind": "windows",
    "sku": {
        "tier": "PremiumV2"
    },
}, {
    "id": "/subscriptions/937bc588-a144-4083-8612-5f9ffbbddb14/resourceGroups/houk-test/providers/Microsoft.Web/serverfarms/houk-fa-plan",
    "location": "East US",
    "kind": "linux",
    "sku": {
        "tier": "Basic"
    },
}]
{
"id": "/subscriptions/937bc588-a144-4083-8612-5f9ffbbddb14/resourceGroups/Cupertino-MVP/providers/Microsoft.Web/serverFarms/ASP-CupertinoMVP-be59",
"location": "centralus",
"kind": "app",
"sku": {
 "tier": "Standard"
}
}

If Location not provided, use the most popular Azure Regions

If Runtime not provided,  choose from below that best matches request
{
  "linux": [
    "DOTNETCORE:7.0",
    "DOTNETCORE:6.0",
    "NODE:18-lts",
    "NODE:16-lts",
    "NODE:14-lts",
    "PYTHON:3.11",
    "PYTHON:3.10",
    "PYTHON:3.9",
    "PYTHON:3.8",
    "PYTHON:3.7",
    "PHP:8.2",
    "PHP:8.1",
    "PHP:8.0",
    "RUBY:2.7",
    "JAVA:17-java17",
    "JAVA:11-java11",
    "JAVA:8-jre8",
    "JBOSSEAP:7-java11",
    "JBOSSEAP:7-java8",
    "TOMCAT:10.0-java17",
    "TOMCAT:10.0-java11",
    "TOMCAT:10.0-jre8",
    "TOMCAT:9.0-java17",
    "TOMCAT:9.0-java11",
    "TOMCAT:9.0-jre8",
    "TOMCAT:8.5-java11",
    "TOMCAT:8.5-jre8",
    "GO:1.19"
  ],
  "windows": [
    "dotnet:7",
    "dotnet:6",
    "ASPNET:V4.8",
    "ASPNET:V3.5",
    "NODE:18LTS",
    "NODE:16LTS",
    "NODE:14LTS",
    "java:1.8:Java SE:8",
    "java:11:Java SE:11",
    "java:17:Java SE:17",
    "java:1.8:TOMCAT:10.0",
    "java:11:TOMCAT:10.0",
    "java:17:TOMCAT:10.0",
    "java:1.8:TOMCAT:9.0",
    "java:11:TOMCAT:9.0",
    "java:17:TOMCAT:9.0",
    "java:1.8:TOMCAT:8.5",
    "java:11:TOMCAT:8.5",
    "java:17:TOMCAT:8.5"
  ]
}

"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
category_and_resource_response_1 = get_completion_from_messages(messages)
print(category_and_resource_response_1)

{
    "id": "/subscriptions/937bc588-a144-4083-8612-5f9ffbbddb14/resourceGroups/Cupertino-MVP/providers/Microsoft.Web/sites/app-abcdefghij?api-version=2022-03-01",
    "payload": {
        "location": "centralus",
        "properties": {
            "serverFarmId": "/subscriptions/937bc588-a144-4083-8612-5f9ffbbddb14/resourceGroups/houk-test/providers/Microsoft.Web/serverfarms/houk-fa-plan",
            "reserved": false,
            "isXenon": false,
            "hyperV": false,
            "siteConfig": {
                "netFrameworkVersion": "v4.6",
                "linuxFxVersion": "DOTNETCORE:6.0",
                "appSettings": [],
                "alwaysOn": true,
                "localMySqlEnabled": false,
                "http20Enabled": true
            },
            "scmSiteAlsoStopped": false,
            "httpsOnly": false
        }
    }
}
